In [1]:
using Revise



In [2]:
using Pkg


Pkg.activate("C:/Users/Pasqu/Documents/julia_projects/LaplaceRedux.jl/docs")
Pkg.status()
using LaplaceRedux
using MLJBase
using Random
using DataFrames
using Flux

  Activating project at `C:\Users\Pasqu\Documents\julia_projects\LaplaceRedux.jl\docs`


Status `C:\Users\Pasqu\Documents\julia_projects\LaplaceRedux.jl\docs\Project.toml`
  [324d7699] CategoricalArrays v0.10.8
⌃ [a93c6f00] DataFrames v1.6.1
⌃ [31c24e10] Distributions v0.25.111
⌃ [e30172f5] Documenter v1.6.0
⌃ [587475ba] Flux v0.14.19
  [c52c1a26] LaplaceRedux v1.1.1 `C:\Users\Pasqu\Documents\julia_projects\LaplaceRedux.jl`
⌅ [094fc8d1] MLJFlux v0.5.1
  [e80e1ace] MLJModelInterface v1.11.0
  [3bd65402] Optimisers v0.3.3
  [91a5bcdd] Plots v1.40.8
  [ce6b1742] RDatasets v0.7.7
  [860ef19b] StableRNGs v1.0.2
  [2913bbd2] StatsBase v0.34.3
  [bd369af6] Tables v1.12.0
⌃ [bd7198b4] TaijaPlotting v1.2.0
  [592b5752] Trapz v2.0.3
⌃ [e88e6eb3] Zygote v0.6.70
⌃ [02a925ec] cuDNN v1.3.2
  [9a3f8284] Random
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
X, y = make_regression(100, 2; noise=0.5, sparse=0.2, outliers=0.1)

(Tables.MatrixTable{Matrix{Float64}} with 100 rows, 2 columns, and schema:
 :x1  Float64
 :x2  Float64, [-0.15053240354230857, -0.16143107735113107, -0.28104782384528254, 0.8905842690519058, -0.2716955057136559, 0.9606721208381163, 0.14403243794060133, 0.13743002853667605, 0.820641892942472, 0.2270783932443115  …  0.4639933046961763, 0.30449384622096687, 0.2744588755171263, -31.785240173822324, 2.58951832655098, 0.10969223924903307, 0.1600255529817666, 0.5913011997917647, -0.39253898214541977, -0.7247243478011852])

In [4]:
using Tables
# Get the number of rows
num_rows = Tables.rows(X) |> length

# Get the number of columns
num_columns = Tables.columnnames(X) |> length

# Display the dimensions
println("Number of rows: ", num_rows)
println("Number of columns: ", num_columns)

Number of rows: 100
Number of columns: 2


In [5]:
using Flux
# Define the model
flux_model = Chain(
    Dense(2, 10, relu),
    Dense(10, 10, relu),
    Dense(10, 1)
)

Chain(
  Dense(2 => 10, relu),                 # 30 parameters
  Dense(10 => 10, relu),                # 110 parameters
  Dense(10 => 1),                       # 11 parameters
)                   # Total: 6 arrays, 151 parameters, 988 bytes.

In [6]:

# Create an instance of LaplaceRegressor with the Flux model
laplace_regressor = LaplaceRegressor(
    model = flux_model,
    subset_of_weights = :all,
    subnetwork_indices = nothing,
    hessian_structure = :full,
    backend = :GGN,
    σ = 1.0,
    μ₀ = 0.0,
    P₀ = nothing,
    fit_prior_nsteps = 100
)

# Display the LaplaceRegressor object
laplace_regressor

LaplaceRegressor(
  model = Chain(Dense(2 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), 
  flux_loss = Flux.Losses.mse, 
  optimiser = Adam(0.001, (0.9, 0.999), 1.0e-8), 
  epochs = 1000, 
  batch_size = 32, 
  subset_of_weights = :all, 
  subnetwork_indices = nothing, 
  hessian_structure = :full, 
  backend = :GGN, 
  σ = 1.0, 
  μ₀ = 0.0, 
  P₀ = nothing, 
  fit_prior_nsteps = 100)

In [7]:
typeof(X)

Tables.MatrixTable{Matrix{Float64}}

In [8]:
using MLJ, DataFrames
X, y = make_blobs(100, 4; centers=2, cluster_std=[1.0, 3.0 ])
dfBlobs = DataFrame(X)
dfBlobs.y = y
first(dfBlobs, 17)

Row,x1,x2,x3,x4,y
,Float64,Float64,Float64,Float64,Cat…
1,5.89419,12.9979,6.18512,8.99286,2
2,-8.14834,6.23246,-1.68497,8.96905,1
3,-4.88229,5.35276,-0.45876,8.20756,1
4,4.02585,6.94769,13.4032,-0.0419223,2
5,-5.53635,6.55656,-1.67063,8.77041,1
6,-6.61858,4.65032,-1.15198,8.34897,1
7,10.2344,11.4278,13.0544,8.53025,2
8,-6.05052,8.12027,-3.68708,8.78732,1
9,-5.06769,4.8631,-3.58346,8.41371,1


In [9]:
using Flux
# Define the model
flux_model = Chain(
    Dense(4, 10, relu),
    Dense(10, 10, relu),
    Dense(10, 2)
)

Chain(
  Dense(4 => 10, relu),                 # 50 parameters
  Dense(10 => 10, relu),                # 110 parameters
  Dense(10 => 2),                       # 22 parameters
)                   # Total: 6 arrays, 182 parameters, 1.086 KiB.

In [10]:
# Create an instance of LaplaceRegressor with the Flux model
laplace_classifier = LaplaceClassifier(
    model = flux_model,
    subset_of_weights = :all,
    subnetwork_indices = nothing,
    hessian_structure = :full,
    backend = :GGN,
    σ = 1.0,
    μ₀ = 0.0,
    P₀ = nothing,
    fit_prior_nsteps = 100
)

# Display the LaplaceRegressor object
laplace_classifier

LaplaceClassifier(
  model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 2)), 
  flux_loss = Flux.Losses.logitcrossentropy, 
  optimiser = Adam(0.001, (0.9, 0.999), 1.0e-8), 
  epochs = 1000, 
  batch_size = 32, 
  subset_of_weights = :all, 
  subnetwork_indices = nothing, 
  hessian_structure = :full, 
  backend = :GGN, 
  σ = 1.0, 
  μ₀ = 0.0, 
  P₀ = nothing, 
  fit_prior_nsteps = 100, 
  link_approx = :probit)

In [11]:
X_test_reformat,y_test_reformat= MLJBase.reformat(laplace_classifier,X,y)

([5.894190948229072 -8.148338397036424 … 4.6901216310216896 8.754850004556681; 12.99791729188348 6.232458473633034 … 10.904369405648966 12.187158715203248; 6.185122057812649 -1.6849673572899138 … 10.09295616393206 15.616135680604215; 8.992861668498659 8.969052840985539 … 8.673387526088336 3.444822170655238], (Bool[1 0 … 1 1; 0 1 … 0 0], CategoricalArrays.CategoricalValue{Int64, UInt32} 2))

In [12]:
y_test_reformat[1]

2×100 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 1  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  …  1  1  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1
 ⋅  1  1  ⋅  1  1  ⋅  1  1  ⋅  1  ⋅  1     ⋅  ⋅  ⋅  ⋅  1  1  1  1  1  1  ⋅  ⋅

In [13]:
view(y_test_reformat[1],:, [2,3,4])

2×3 view(OneHotMatrix(::Vector{UInt32}), :, [2, 3, 4]) with eltype Bool:
 0  0  1
 1  1  0

In [14]:
view(X_test_reformat, :, [2,3,4])

4×3 view(::Matrix{Float64}, :, [2, 3, 4]) with eltype Float64:
 -8.14834  -4.88229   4.02585
  6.23246   5.35276   6.94769
 -1.68497  -0.45876  13.4032
  8.96905   8.20756  -0.0419223

In [15]:
using MLJ
#DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

using Flux
# Define the model
flux_model = Chain(
    Dense(4, 10, relu),
    Dense(10, 10, relu),
    Dense(10, 3)
)

model = LaplaceClassifier(model=flux_model)

X, y = @load_iris


((sepal_length = [5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9  …  6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9], sepal_width = [3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1  …  3.1, 3.1, 2.7, 3.2, 3.3, 3.0, 2.5, 3.0, 3.4, 3.0], petal_length = [1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5  …  5.6, 5.1, 5.1, 5.9, 5.7, 5.2, 5.0, 5.2, 5.4, 5.1], petal_width = [0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1  …  2.4, 2.3, 1.9, 2.3, 2.5, 2.3, 1.9, 2.0, 2.3, 1.8]), CategoricalArrays.CategoricalValue{String, UInt32}["setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa"  …  "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica"])

In [16]:
levels(y)

3-element Vector{String}:
 "setosa"
 "versicolor"
 "virginica"

In [17]:
mach = machine(model, X, y) |> MLJBase.fit!

┌ Info: Training machine(LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:499
┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(4 => 10, relu)
│   summary(x) = 4×32 Matrix{Float64}
└ @ Flux C:\Users\Pasqu\.julia\packages\Flux\HBF2N\src\layers\stateless.jl:60


Epoch 100: Loss: 4.042439937591553 
Epoch 200: Loss: 3.533539354801178 
Epoch 300: Loss: 3.2006053030490875 
Epoch 400: Loss: 2.9201304614543915 
Epoch 500: Loss: 2.647124856710434 
Epoch 600: Loss: 2.3577273190021515 
Epoch 700: Loss: 2.0284294933080673 
Epoch 800: Loss: 1.6497879326343536 
Epoch 900: Loss: 1.3182911276817322 
Epoch 1000: Loss: 1.077766239643097 


trained Machine; caches model-specific representations of data
  model: LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …)
  args: 
    1:	Source @378 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @187 ⏎ AbstractVector{Multiclass{3}}


In [18]:
Xnew = (sepal_length = [6.4, 7.2, 7.4],
        sepal_width = [2.8, 3.0, 2.8],
        petal_length = [5.6, 5.8, 6.1],
        petal_width = [2.1, 1.6, 1.9],)
yhat = MLJBase.predict(mach, Xnew) # probabilistic predictions
predict_mode(mach, Xnew)   # point predictions
pdf.(yhat, "virginica")    # probabilities for the "verginica" class

3-element Vector{Float64}:
 0.9488185211171959
 0.7685919895442062
 0.9454937120287679

In [19]:
using Random
cv = CV(nfolds=3)
CV(
  nfolds = 3,
  shuffle = false,
  rng = Random.default_rng())

CV(
  nfolds = 3, 
  shuffle = false, 
  rng = TaskLocalRNG())

In [20]:
MLJBase.fitted_params(mach)

(μ = Float32[0.36796558, -0.590969, -0.0005966219, -0.1918786, -0.21046183, 0.33172563, -0.6454487, -0.43057427, 0.09267368, -0.31252357  …  0.31747338, -0.42893136, -0.036560666, 0.12848923, -0.022290554, -0.17562295, 0.10335993, 2.0584264, -0.27860576, -0.9749556],
 H = [19221.80854034424 0.0 … 175.01133728027344 416.5687526464462; 0.0 0.0 … 0.0 0.0; … ; 175.01133728027344 0.0 … 150.0 0.0; 416.5687526464462 0.0 … 0.0 150.0],
 P = [19222.80854034424 0.0 … 175.01133728027344 416.5687526464462; 0.0 1.0 … 0.0 0.0; … ; 175.01133728027344 0.0 … 151.0 0.0; 416.5687526464462 0.0 … 0.0 151.0],
 Σ = [0.4942412474220449 0.0 … 0.00012692197672594825 -0.00029493662610333626; 0.0 1.0 … -0.0 -0.0; … ; 0.00012692197672495198 0.0 … 0.019500175331854216 2.549733600297664e-5; -0.000294936626108007 0.0 … 2.5497336002974752e-5 0.019426448013349775],
 n_data = 160,
 n_params = 193,
 n_out = 3,
 loss = 76.21374633908272,)

In [21]:
model.epochs= 1500
uff=MLJBase.fit!(mach)

Epoch 1100: Loss: 0.9005963057279587 
Epoch 1200: Loss: 0.7674828618764877 
Epoch 1300: Loss: 0.6668129041790962 

┌ Info: Updating machine(LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:500



Epoch 1400: Loss: 0.5900264084339142 
Epoch 1500: Loss: 0.5311783701181412 


trained Machine; caches model-specific representations of data
  model: LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …)
  args: 
    1:	Source @378 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @187 ⏎ AbstractVector{Multiclass{3}}


In [22]:
model.fit_prior_nsteps = 200
uff=MLJBase.fit!(mach)

 updating only the laplace optimization part


┌ Info: Updating machine(LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:500


trained Machine; caches model-specific representations of data
  model: LaplaceClassifier(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 3)), …)
  args: 
    1:	Source @378 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @187 ⏎ AbstractVector{Multiclass{3}}


In [23]:
using MLJ
#LaplaceRegressor = @load LaplaceRegressor pkg=LaplaceRedux
flux_model = Chain(
    Dense(4, 10, relu),
    Dense(10, 10, relu),
    Dense(10, 1)
)
model = LaplaceRegressor(model=flux_model)

X, y = make_regression(100, 4; noise=0.5, sparse=0.2, outliers=0.1)
mach = machine(model, X, y) 
uff=MLJBase.fit!(mach)
Xnew, _ = make_regression(3, 4; rng=123)
yhat = MLJBase.predict(mach, Xnew) # probabilistic predictions
MLJBase.predict_mode(mach, Xnew)   # point predictions



┌ Info: Training machine(LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


Epoch 100: Loss: 123.99619626934783 
Epoch 200: Loss: 110.70134709925446 
Epoch 300: Loss: 101.54836914035226 
Epoch 400: Loss: 93.59849316803738 
Epoch 500: Loss: 85.18775669356168 
Epoch 600: Loss: 75.76182416873147 
Epoch 700: Loss: 67.35201676247976 
Epoch 800: Loss: 61.22682561405186 
Epoch 900: Loss: 56.388587609986764 
Epoch 1000: Loss: 51.8525101259686 


1×3 Matrix{Float64}:
 -2.0295  -1.03661  1.29323

In [24]:
MLJBase.fitted_params(mach)

(μ = Float32[1.0187618, 1.1801981, 1.5047036, 0.09397529, -1.0688609, -0.59974676, 1.2977643, -1.8645017, -0.6493797, 0.8598072  …  -3.1050463, -2.267896, 1.018275, -2.0316908, 0.72548866, -2.7309833, -2.1938443, 0.8200029, 1.2517836, 0.3313799],
 H = [8124.873962402344 1238.1352081298828 … 1980.7237396240234 324.216007232666; 1238.1352081298828 496.2319145202637 … 451.8493309020996 62.39444828033447; … ; 1980.7237396240234 451.8493309020996 … 1277.6469421386719 212.83043670654297; 324.216007232666 62.39444828033447 … 212.83043670654297 100.0],
 P = [8125.873962402344 1238.1352081298828 … 1980.7237396240234 324.216007232666; 1238.1352081298828 497.2319145202637 … 451.8493309020996 62.39444828033447; … ; 1980.7237396240234 451.8493309020996 … 1278.6469421386719 212.83043670654297; 324.216007232666 62.39444828033447 … 212.83043670654297 101.0],
 Σ = [0.08818398027482557 0.01068994964485469 … -0.028911073548363073 0.0029380249323247053; 0.01068994964484666 0.3740392430965252 … -0.03321343

In [25]:
a= MLJBase.training_losses(mach)

1000-element Vector{Any}:
 132.4757012476366
 132.34841820836348
 132.2594675536756
 132.1764167781873
 132.09720400291872
 132.02269947440837
 131.95014241005182
 131.87929837827625
 131.8104252672236
 131.74319014461662
   ⋮
  52.175198365270234
  52.148401306572445
  52.10149557862212
  52.032551867920525
  51.99690913470439
  51.994054497037524
  51.97321907106968
  51.932579915942064
  51.8525101259686

In [26]:
model.epochs= 1500
uff=MLJBase.fit!(mach)



┌ Info: Updating machine(LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:500


Epoch 1100: Loss: 47.37399118448011 
Epoch 1200: Loss: 42.67007994813126 
Epoch 1300: Loss: 38.057605481177234 
Epoch 1400: Loss: 33.846777755430416 
Epoch 1500: Loss: 29.514834265471144 


trained Machine; caches model-specific representations of data
  model: LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …)
  args: 
    1:	Source @447 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @645 ⏎ AbstractVector{Continuous}


In [27]:
MLJBase.training_losses(mach)

1500-element Vector{Any}:
 132.4757012476366
 132.34841820836348
 132.2594675536756
 132.1764167781873
 132.09720400291872
 132.02269947440837
 131.95014241005182
 131.87929837827625
 131.8104252672236
 131.74319014461662
   ⋮
  29.979098347946184
  29.896050775635096
  29.83609799111873
  29.763058304997937
  29.69908817371154
  29.646864259231556
  29.623977677653713
  29.549562760589527
  29.514834265471144

In [28]:
model.epochs= 1200
uff=MLJBase.fit!(mach)

The number of epochs inserted is lower than the number of epochs already been trained. No update is necessary


┌ Info: Updating machine(LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:500


trained Machine; caches model-specific representations of data
  model: LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …)
  args: 
    1:	Source @447 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @645 ⏎ AbstractVector{Continuous}


In [29]:
model.fit_prior_nsteps = 200
uff=MLJBase.fit!(mach)

 updating only the laplace optimization part


┌ Info: Updating machine(LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …), …).
└ @ MLJBase C:\Users\Pasqu\.julia\packages\MLJBase\7nGJF\src\machines.jl:500


trained Machine; caches model-specific representations of data
  model: LaplaceRegressor(model = Chain(Dense(4 => 10, relu), Dense(10 => 10, relu), Dense(10 => 1)), …)
  args: 
    1:	Source @447 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @645 ⏎ AbstractVector{Continuous}


In [30]:
MLJBase.selectrows(model, I, Xmatrix, y) = (view(Xmatrix, :, I), view(y[1], I))

In [31]:
#MLJBase.evaluate(model, X, y, resampling=cv, measure=l2, verbosity=0)

In [32]:
evaluate!(mach, resampling=cv, measure=l2, verbosity=0)

BoundsError: BoundsError: attempt to access 4×100 Matrix{Float64} at index [[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], 1:100]

In [33]:
#flux_model = Chain(
    #Dense(4, 10, relu),
    #Dense(10, 10, relu),
    #Dense(10, 1))


#nested_flux_model = Chain(
    #Chain(Dense(10, 5, relu), Dense(5, 5, relu)),
    #Chain(Dense(5, 3, relu), Dense(3, 2)))
#model = LaplaceRegressor()
#model.model= nested_flux_model

#copy_model= deepcopy(model)

#copy_model.epochs= 2000
#copy_model.optimiser = Descent()
#MLJBase.is_same_except(model , copy_model,:epochs )